In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [16]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [17]:
wiki.name

'wikipedia'

In [11]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vector_db = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E7392C1850>, search_kwargs={})

In [12]:
from langchain.tools.retriever import create_retriever_tool

retrieval_tool = create_retriever_tool(retriever, "langsmith_search", "Search for information about langsmith")

In [13]:
retrieval_tool.name

'langsmith_search'

In [15]:
# Arxiv tool

from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(arxiv_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [20]:
tools = [arxiv, wiki, retrieval_tool]
tools

[ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000)),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Practice\\RAG\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001E76B900040>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E7392C1850>, search_kwargs

In [22]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
import os
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)


In [23]:
from langchain import hub

# Get the prompt to use - you can modify this
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [25]:
# Agents

from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [27]:
# Agent Executor

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [30]:
response = agent_executor.invoke({"input": "What is the use of quantum computer"})



> Entering new AgentExecutor chain...
Quantum computers are a type of computing device that use principles of quantum mechanics to perform operations on data. They have the potential to solve certain problems much faster than classical computers. Some of the key uses and advantages of quantum computers include:

1. **Speed**: Quantum computers can solve complex problems exponentially faster than classical computers for certain tasks. This speed advantage is particularly useful for tasks like cryptography, optimization problems, and simulations.

2. **Quantum Cryptography**: Quantum computers can enhance security through quantum cryptography, which relies on the principles of quantum mechanics to create secure communication channels.

3. **Optimization**: Quantum computers can optimize complex systems and processes more efficiently than classical computers. This is valuable in industries like finance, logistics, and manufacturing.

4. **Machine Learning**: Quantum computers have the p

In [32]:
print(response['output'])

Quantum computers are a type of computing device that use principles of quantum mechanics to perform operations on data. They have the potential to solve certain problems much faster than classical computers. Some of the key uses and advantages of quantum computers include:

1. **Speed**: Quantum computers can solve complex problems exponentially faster than classical computers for certain tasks. This speed advantage is particularly useful for tasks like cryptography, optimization problems, and simulations.

2. **Quantum Cryptography**: Quantum computers can enhance security through quantum cryptography, which relies on the principles of quantum mechanics to create secure communication channels.

3. **Optimization**: Quantum computers can optimize complex systems and processes more efficiently than classical computers. This is valuable in industries like finance, logistics, and manufacturing.

4. **Machine Learning**: Quantum computers have the potential to enhance machine learning alg